In [12]:
from ensemble_boxes import nms, soft_nms, non_maximum_weighted, weighted_boxes_fusion

import os
import shutil
import pandas as pd
import numpy as np

## CSV file load.

In [13]:
save_paths = [
    "[0.5313] VFNet_instaboost",
    "[0.5323] VFNet_insatboost_mosaic",
    "[0.5397] VFNet_instaboost_TTA",
]
lb_scores = [1,1,1]

data_dfs = [pd.read_csv(os.path.join("save_dir", "Weighted_Boxes_Fusion", f"{save_path}.csv")) for save_path in save_paths]

## Helper function define.

In [14]:
def get_values(predicted):
    predicted = predicted.split(" ")
    
    labels = []
    scores = []
    boxes = []
    
    for i in range(0, len(predicted) - 1, 6):
        label = int(predicted[i])
        score = float(predicted[i + 1])
        box = list(map(lambda x: float(x) / 512, predicted[i + 2:i + 6]))
        
        labels.append(label)
        scores.append(score)
        boxes.append(box)
        
    return boxes, scores, labels

## Weighted boxes fusion.

In [15]:
iou_thr = 0.6
skip_box_thr = 0.0001

In [16]:
wbf_results = []

for idx in range(len(data_dfs[0])):
    boxes_ = []
    scores_ = []
    labels_ = []
    for df_idx in range(len(data_dfs)):
        predicted = data_dfs[df_idx].iloc[idx, 0]
        box, score, label = get_values(predicted)
        boxes_.append(box)
        scores_.append(score)
        labels_.append(label)
        
    wbf_box, wbf_score, wbf_label = weighted_boxes_fusion(boxes_, scores_, labels_, weights=lb_scores, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    wbf_result = [[int(wbf_label_)] + [wbf_score_] + list(map(lambda x: x * 512, wbf_box_)) for wbf_box_, wbf_score_, wbf_label_ in zip(wbf_box, wbf_score, wbf_label)]
    wbf_result = sorted(wbf_result, key=lambda x: x[1], reverse=True)
    wbf_result = sorted(wbf_result, key=lambda x: x[0])
    
    wbf_results.append(wbf_result)

## Save to CSV file.

In [17]:
image_names = data_dfs[0].iloc[:, 1]

prediction_strings = []
file_names = []

for wbf_result, image_name in zip(wbf_results, image_names):
    prediction_string = ""
    for wbf_result_ in wbf_result:
        label = str(wbf_result_[0])
        score = str(wbf_result_[1])
        box = " ".join(list(map(lambda x: str(x), wbf_result_[2:])))
        prediction_string += label + " " + score + " " + box + " "
    
    prediction_strings.append(prediction_string)
    file_names.append(image_name)
    
submission = pd.DataFrame()
submission["PredictionString"] = prediction_strings
submission["image_id"] = file_names
submission.to_csv(os.path.join("save_dir", "Weighted_Boxes_Fusion", f"submission_wbf_ensemble.csv"), index=None)
submission.head()

,PredictionString,image_id
0,0 0.09945263713598251 165.0948028564453 75.666...,batch_01_vt/0021.jpg
1,0 0.16466228663921356 284.54278564453125 342.5...,batch_01_vt/0028.jpg
2,0 0.10302789509296417 235.49957275390625 460.9...,batch_01_vt/0031.jpg
3,0 0.0994613915681839 293.2613830566406 156.553...,batch_01_vt/0032.jpg
4,0 0.11256132666666667 176.64763 208.54279 288....,batch_01_vt/0070.jpg
